In [1]:
RunningDB = True
import aerospike
import sys
import json
import time
import random
import statistics
import pprint

import math

In [2]:
print("Current Python version: ", sys.version_info[0],".",sys.version_info[1],".",sys.version_info[2])

Current Python version:  3 . 6 . 8


In [3]:
_useAllTimes = False
_print_raw = False

mport the Table class used to build Table schema

In [4]:
from Private_Pkgs.Table import Table
from Private_Pkgs.DataRecordingUtils import drutils


from IPython.core.debugger import set_trace

In [5]:
def BuildTables():
    MechTable = Table("test_DB_SEF", "mechanics")
    MechTable.AddCol("mech_num")
    MechTable.AddCol("mech_name")
    MechTable.AddCol("mech_salary")
    MechTable.AddCol("mech_specialty")
    MechTable.AddPK("mech_num")
    if MechTable.VerifyFKrefsCompletePK():
        print(MechTable.TableName, "has complete FK to PK references (or none exist in table)")

    CarTable = Table("test_DB_SEF", "cars")
    CarTable.AddCol("car_num")
    CarTable.AddCol("car_model")
    CarTable.AddCol("car_year")
    CarTable.AddPK("car_num")
    if CarTable.VerifyFKrefsCompletePK():
        print(CarTable.TableName, "has complete FK to PK references (or none exist in table)")

    ReprTable = Table("test_DB_SEF", "repairs")
    ReprTable.AddCol("mech_num")
    ReprTable.AddCol("car_num")
    ReprTable.AddCol("repair_date")
    ReprTable.AddPK("mech_num")
    ReprTable.AddPK("car_num")
    ReprTable.AddPK("repair_date")
    ReprTable.AddFK("mech_num", "mechanics", "mech_num")
    ReprTable.AddFK("car_num", "cars", "car_num")
    if ReprTable.VerifyFKrefsCompletePK():
        print(ReprTable.TableName, "has complete FK to PK references (or none exist in table)")
    return True

In [6]:
def InsertData():
    rprTable_times = []
    insertAll_times = {}
    MechTable = Table._registry[0]
    CarTable = Table._registry[1]
    RepairTable = Table._registry[2]
    for rowNum in range(0,int(NumberRows)):
        thisRow = [rowNum,"someName", "someSalary", "someSpecialty"]
        MechTable.Insert(thisRow)
        thisRow = [rowNum,"someCar", "someYear"]
        CarTable.Insert(thisRow)

    NumberHashesPerFK = 5

    for rowNum in range(0,int(NumberRows)):
        mechNum = math.floor(rowNum/NumberHashesPerFK)
        carNum = math.floor(rowNum/NumberHashesPerFK)
        thisRow =  [mechNum,carNum,"someRepairDate"+str(rowNum)] #The composite PK. Both rowNums are FKs
        t0 = time.time()
        RepairTable.Insert(thisRow)
        t1 = time.time()
        rprTable_times.append((t1 - t0) * 1000) # Times are in S so *1000 makes units mS
    insertAll_times["RepairTable"] = rprTable_times
    return(insertAll_times)

In [7]:
def UpdateData():
    rprTable_times = []
    All_times = {}
    MechTable = Table._registry[0]
    CarTable = Table._registry[1]
    RepairTable = Table._registry[2]
    #Setup a random, no repeating sequence
    random.seed(42)
    row_seq = list(range(0, NumberRows))
    random.shuffle(row_seq)
    for idx in range(0, int(NumberRows/2)):
        thisRow =[]
        thisRow.append(row_seq[idx]) #1st element of PK
        thisRow.append(row_seq[idx]) #2nd element of PK
        thisRow.append(str("someRepairDateX"+str(row_seq[idx]))) # 3rd element of PK that will make the PK unique
        t0 = time.time()
        RepairTable.Update(thisRow)
        #print("TableA update: ", thisRow)
        t1 = time.time()
        rprTable_times.append((t1 - t0) * 1000) # Times are in S so *1000 makes units mS
        
    All_times["RepairTable"] = rprTable_times
    return(All_times)

In [8]:
def DeleteData():
    mechTable_times = []
    carTable_times = []
    rprTable_times = []
    All_times = {}
    MechTable = Table._registry[0]
    CarTable = Table._registry[1]
    RepairTable = Table._registry[2]
    #Setup a random, no repeating sequence
    random.seed(42)
    row_seq = list(range(0, NumberRows))
    random.shuffle(row_seq)
    
    for idx in range(0, int(NumberRows/2)):
        t0 = time.time()
        RepairTable.Delete([row_seq[idx],row_seq[idx],"someRepairDate"])
        t1 = time.time()
        rprTable_times.append((t1 - t0) * 1000) # Times are in S so *1000 makes units mS
        
        t0 = time.time()
        MechTable.Delete([row_seq[idx]])
        t1 = time.time()
        mechTable_times.append((t1 - t0) * 1000) # Times are in S so *1000 makes units mS
        
        t0 = time.time()
        CarTable.Delete([row_seq[idx]])
        t1 = time.time()
        carTable_times.append((t1 - t0) * 1000) # Times are in S so *1000 makes units mS
        
    All_times["RepairTable"] = rprTable_times
    All_times["MechTable"] = mechTable_times
    All_times["CarTable"] = carTable_times
    return(All_times)

In [9]:
def report_times(insertAll_times, silent = False):
    BenchTimes = {}
    benchdata.rcrd_data(str("Statistics for Insert with {} rows, Verify Constraints set to: {}").format(NumberRows,Table.GetVerifyConstraints()))
    for dataKey in insertAll_times:
        insert_times = insertAll_times[dataKey]
        if _print_raw:
            print("Raw insert times for:", dataKey)
            pp = pprint.PrettyPrinter(indent=4)
            pp.pprint(insert_times)
        BenchTimes[dataKey] = (NumberRows, Table.GetVerifyConstraints(), 
                         statistics.median(insert_times),
                         statistics.mean(insert_times),
                         statistics.stdev(insert_times))
        print("Statistics for: ", dataKey)
        benchdata.rcrd_data("\tMEDIAN"+str(statistics.median(insert_times))+" ms")
        benchdata.rcrd_data("\tMEAN  "+str(statistics.mean(insert_times))+" ms")
        benchdata.rcrd_data("\tSTDEV "+str(statistics.stdev(insert_times))+" ms")
    return BenchTimes

In [10]:
def report_statistics(withConBenchTimeAll, noConBenchTimeAll):
    for withConKey, noConKey in zip(withConBenchTimeAll, noConBenchTimeAll):
        print("Statistics for: ", withConKey)
        withConBenchTime = withConBenchTimeAll[withConKey]
        noConBenchTime = noConBenchTimeAll[noConKey]
        benchdata.rcrd_data()
        benchdata.rcrd_data("\tWith Constraints:\t\tWithout Constraints\t\tPercent Difference, {} rows".format(noConBenchTime[0]))
        benchdata.rcrd_data("MEDIAN:\t{} \t\t{} ms\t\t{}".format(withConBenchTime[2],noConBenchTime[2],
             (withConBenchTime[2]-noConBenchTime[2])/noConBenchTime[2]*100))
        benchdata.rcrd_data("MEAN:\t{} \t\t{} ms\t\t{}".format(withConBenchTime[3],noConBenchTime[3],
             (withConBenchTime[3]-noConBenchTime[2])/noConBenchTime[3]*100))
        benchdata.rcrd_data("STDEV:\t{} \t\t{} ms\t\t{}".format(withConBenchTime[4],noConBenchTime[4],
             (withConBenchTime[4]-noConBenchTime[4])/noConBenchTime[4]*100))
        benchdata.rcrd_data()
    return

In [11]:
Table.SetDebugMode(False)
Table.GetDebugMode()

Debug mode is Off


***Open communication with the database***<br>
Specify the IP addresses and ports for the Aerospike cluster

In [12]:
config = {
  'hosts': [ ('18.221.189.247', 3000), ('18.224.137.105', 3000) ]
}
# Create a client and connect it to the cluster
if RunningDB:
    try:
      client = aerospike.client(config).connect()
    except:
      print("failed to connect to the cluster with", config['hosts'])
      sys.exit(1)
else:
    client = FakeAerospike(config) #.connect()

e sure to tell the Table class the name of the client it's talking to

In [13]:
Table.SetTableClient(client)

Info: Table Client Set, connection state is:  True


==============================================<br>
   Start of Main Program Loop<br>
==============================================

In [14]:
if len(sys.argv) == 0:
    NumberRows = input("How many records to BenchMark?")
else:
    NumberRows = sys.argv[1]

In [15]:
#NumberRows = int(NumberRows)
print("Hardcoded number rows!!!!!!!!!!!")
NumberRows = 10
if NumberRows >= 1000:
    NumberRowsStr = str(int(NumberRows/1000))+"K"
else:
    NumberRowsStr = str(NumberRows)

Hardcoded number rows!!!!!!!!!!!


     ==============================================<br>
        Insert Section<br>
     ==============================================<br>
nitialize data reporting

In [16]:
benchdata =drutils("Schema2_Insert_expt_"+NumberRowsStr+"_","s")

Recording data to screen only


In [17]:
Table.SetVerifyConstraints(True)
Table.UseFKTables(True)

***Build the tables***

In [18]:
BuildTables()

mechanics has no FK constraints to verify.
mechanics has complete FK to PK references (or none exist in table)
cars has no FK constraints to verify.
cars has complete FK to PK references (or none exist in table)
thisPK == thisFK refColName:  mech_num mech_num with #PKs found 1
thisPK == thisFK refColName:  car_num car_num with #PKs found 1
repairs has complete FK to PK references (or none exist in table)


True

***Insert priming data, collect Insert times***

In [19]:
insertStartTime = time.time() #insertStartTime/EndTime only use to give an overall time for the insert loop
insertAll_times = InsertData()
insertEndTime = time.time()

### Calculate Time Statistics (Checking constraints during Insert)

In [20]:
withConstraintTimesAll = report_times(insertAll_times)

Verifying Constraints is ENABLED
Statistics for Insert with 10 rows, Verify Constraints set to: True
Verifying Constraints is ENABLED
Statistics for:  RepairTable
	MEDIAN3.6325454711914062 ms
	MEAN  3.8095951080322266 ms
	STDEV 0.3969095839095727 ms


### Remove Benchmark Tables

In [21]:
Table.RemoveAllTables(client, True) #True to wait for confirmation

Removing rows in Table  mechanics
Removing rows in Table  cars
Removing rows in Table  repairs
Removing rows in Table  mech_num_FKTable
Removing rows in Table  car_num_FKTable
Note:  50 records removed


### Rebuild Tables and re-insert data, after disabling Constraints

In [22]:
Table.SetVerifyConstraints(False)
Table.UseFKTables(False)

In [23]:
BuildTables()
insertAll_times = InsertData()
noConstraintTimesAll = report_times(insertAll_times)

mechanics has complete FK to PK references (or none exist in table)
cars has complete FK to PK references (or none exist in table)
repairs has complete FK to PK references (or none exist in table)
PKvals from Insert:  [0] for Table:  mechanics
JSON string: {"mech_num": 0, "mech_name": "someName", "mech_salary": "someSalary", "mech_specialty": "someSpecialty"}
PKvals from Insert:  [0] for Table:  cars
JSON string: {"car_num": 0, "car_model": "someCar", "car_year": "someYear"}
PKvals from Insert:  [1] for Table:  mechanics
JSON string: {"mech_num": 1, "mech_name": "someName", "mech_salary": "someSalary", "mech_specialty": "someSpecialty"}
PKvals from Insert:  [1] for Table:  cars
JSON string: {"car_num": 1, "car_model": "someCar", "car_year": "someYear"}
PKvals from Insert:  [2] for Table:  mechanics
JSON string: {"mech_num": 2, "mech_name": "someName", "mech_salary": "someSalary", "mech_specialty": "someSpecialty"}
PKvals from Insert:  [2] for Table:  cars
JSON string: {"car_num": 2, "c

### Calculate Time Statistics (No constraint verification during Insert)    Close the data object

In [24]:
benchdata.rcrd_data("Total Insert Time: {} seconds, 3 tables".format(insertEndTime - insertStartTime))
report_statistics(withConstraintTimesAll, noConstraintTimesAll)
benchdata.close()
del benchdata

Total Insert Time: 0.057700157165527344 seconds, 3 tables
Statistics for:  RepairTable


	With Constraints:		Without Constraints		Percent Difference, 10 rows
MEDIAN:	3.6325454711914062 		0.6372928619384766 ms		469.9962588851478
MEAN:	3.8095951080322266 		0.6346940994262695 ms		499.815934788325
STDEV:	0.3969095839095727 		0.020641673584784425 ms		1822.8556360960297




     ==============================================<br>
        Update Section<br>
     ==============================================

In [25]:
benchdata =drutils("Schema2_Update_expt_"+NumberRowsStr+"_","s")

Recording data to screen only


# We already have tables that do NOT use constraints or FK Tables ###<br>
herefore, the first run of Update and Delete will be with ***No*** constraints

In [31]:
updateStartTime = time.time() #updateStartTime/EndTime only use to give an overall time for the update loop
updateAll_times = UpdateData()
updateEndTime = time.time()

PKvals from Update:  [7, 7, 'someRepairDateX7'] for Table:  repairs
JSON string: {"mech_num": 7, "car_num": 7, "repair_date": "someRepairDateX7"}
PKvals from Update:  [3, 3, 'someRepairDateX3'] for Table:  repairs
JSON string: {"mech_num": 3, "car_num": 3, "repair_date": "someRepairDateX3"}
PKvals from Update:  [2, 2, 'someRepairDateX2'] for Table:  repairs
JSON string: {"mech_num": 2, "car_num": 2, "repair_date": "someRepairDateX2"}
PKvals from Update:  [8, 8, 'someRepairDateX8'] for Table:  repairs
JSON string: {"mech_num": 8, "car_num": 8, "repair_date": "someRepairDateX8"}
PKvals from Update:  [5, 5, 'someRepairDateX5'] for Table:  repairs
JSON string: {"mech_num": 5, "car_num": 5, "repair_date": "someRepairDateX5"}


In [32]:
noConstraintTimesAll = report_times(updateAll_times)

Verifying Constraints is DISABLED
Statistics for Insert with 10 rows, Verify Constraints set to: False
Verifying Constraints is DISABLED
Statistics for:  RepairTable
	MEDIAN1.5501976013183594 ms
	MEAN  1.83563232421875 ms
	STDEV 0.5648518662484078 ms


**Remove and then Rebuild the Tables with constraints***

In [33]:
Table.RemoveAllTables(client, True) #True to wait for confirmation

Removing rows in Table  mechanics
Removing rows in Table  cars
Removing rows in Table  repairs
Removing rows in Table  mech_num_FKTable
Removing rows in Table  car_num_FKTable
Note:  30 records removed


In [34]:
Table.SetVerifyConstraints(True)
Table.UseFKTables(True)

In [35]:
BuildTables()
InsertData()

mechanics has no FK constraints to verify.
mechanics has complete FK to PK references (or none exist in table)
cars has no FK constraints to verify.
cars has complete FK to PK references (or none exist in table)
refTables in FK list: [<Private_Pkgs.Table.Table object at 0x7f4f0c125cc0>, <Private_Pkgs.Table.Table object at 0x7f4f0c125e80>]
thisPK == thisFK refColName:  mech_num mech_num with #PKs found 1
thisPK == thisFK refColName:  car_num car_num with #PKs found 1
repairs has complete FK to PK references (or none exist in table)
PKvals from Insert:  [0] for Table:  mechanics
JSON string: {"mech_num": 0, "mech_name": "someName", "mech_salary": "someSalary", "mech_specialty": "someSpecialty"}
PKvals from Insert:  [0] for Table:  cars
JSON string: {"car_num": 0, "car_model": "someCar", "car_year": "someYear"}
PKvals from Insert:  [1] for Table:  mechanics
JSON string: {"mech_num": 1, "mech_name": "someName", "mech_salary": "someSalary", "mech_specialty": "someSpecialty"}
PKvals from Ins

{'RepairTable': [3.715038299560547,
  5.064725875854492,
  3.9565563201904297,
  3.7240982055664062,
  3.875255584716797,
  3.612995147705078,
  3.5767555236816406,
  3.538370132446289,
  3.6797523498535156,
  3.7012100219726562]}

In [36]:
updateStartTime = time.time() #updateStartTime/EndTime only use to give an overall time for the update loop
updateAll_times = UpdateData()
updateEndTime = time.time()

PKvals from Update:  [7, 7, 'someRepairDateX7'] for Table:  repairs
Update failed! PK value doesn't currently exist in Table:  repairs
PKvals from Update:  [3, 3, 'someRepairDateX3'] for Table:  repairs
Update failed! PK value doesn't currently exist in Table:  repairs
PKvals from Update:  [2, 2, 'someRepairDateX2'] for Table:  repairs
Update failed! PK value doesn't currently exist in Table:  repairs
PKvals from Update:  [8, 8, 'someRepairDateX8'] for Table:  repairs
Update failed! PK value doesn't currently exist in Table:  repairs
PKvals from Update:  [5, 5, 'someRepairDateX5'] for Table:  repairs
Update failed! PK value doesn't currently exist in Table:  repairs


In [ ]:
withConstraintTimesAll = report_times(updateAll_times)

In [ ]:
benchdata.rcrd_data("Total Update Time: {} seconds, 2 tables".format(updateEndTime - updateStartTime))
report_statistics(withConstraintTimesAll, noConstraintTimesAll)

In [44]:
benchdata.close()
del benchdata

emove all tables and rebuild them for the delete.

In [45]:
Table.RemoveAllTables(client, True) #True to wait for confirmation

Removing rows in Table  mechanics
Removing rows in Table  cars
Removing rows in Table  repairs
Removing rows in Table  mech_num_FKTable
Removing rows in Table  car_num_FKTable
Note:  25 records removed


     ==============================================<br>
        Delete Section<br>
     ==============================================

In [46]:
benchdata =drutils("Schema2_Delete_expt_"+NumberRowsStr+"_","s")

Recording data to screen only


In [47]:
Table.SetVerifyConstraints(True)
Table.UseFKTables(True)

In [48]:
BuildTables()
InsertData()

mechanics has no FK constraints to verify.
mechanics has complete FK to PK references (or none exist in table)
cars has no FK constraints to verify.
cars has complete FK to PK references (or none exist in table)
refTables in FK list: [<Private_Pkgs.Table.Table object at 0x7f4f0c0ae828>, <Private_Pkgs.Table.Table object at 0x7f4f0c0ae978>]
thisPK == thisFK refColName:  mech_num mech_num with #PKs found 1
thisPK == thisFK refColName:  car_num car_num with #PKs found 1
repairs has complete FK to PK references (or none exist in table)
PKvals from Insert:  [0] for Table:  mechanics
JSON string: {"mech_num": 0, "mech_name": "someName", "mech_salary": "someSalary", "mech_specialty": "someSpecialty"}
PKvals from Insert:  [0] for Table:  cars
JSON string: {"car_num": 0, "car_model": "someCar", "car_year": "someYear"}
PKvals from Insert:  [1] for Table:  mechanics
JSON string: {"mech_num": 1, "mech_name": "someName", "mech_salary": "someSalary", "mech_specialty": "someSpecialty"}
PKvals from Ins

{'RepairTable': [3.543376922607422,
  3.4296512603759766,
  3.682374954223633,
  3.381013870239258,
  3.462076187133789,
  3.4584999084472656,
  3.538846969604492,
  3.4914016723632812,
  3.456592559814453,
  3.4830570220947266]}

In [49]:
deleteStartTime = time.time() #deleteStartTime/EndTime only use to give an overall time for the delete loop
deleteAll_times = DeleteData()
deleteEndTime = time.time()

[7, 7, 'someRepairDate']  exists in the following row (VerifyUniquePKval):
{'mech_num': 7, 'car_num': 7, 'repair_date': 'someRepairDate'}
****This table has FK refs (it's a parent): repairs
****Updating FKtable  mech_num_FKTable
****Updating FKtable  car_num_FKTable
****Removing record  [7, 7, 'someRepairDate']
[7]  exists in the following row (VerifyUniquePKval):
{'mech_num': 7, 'mech_name': 'someName', 'mech_salary': 'someSalary', 'mech_specialty': 'someSpecialty'}
****This table has no FK refs, checking for references: mechanics
****No references found, removing record [7]
[7]  exists in the following row (VerifyUniquePKval):
{'car_num': 7, 'car_model': 'someCar', 'car_year': 'someYear'}
****This table has no FK refs, checking for references: cars
****No references found, removing record [7]
[3, 3, 'someRepairDate']  exists in the following row (VerifyUniquePKval):
{'mech_num': 3, 'car_num': 3, 'repair_date': 'someRepairDate'}
****This table has FK refs (it's a parent): repairs
****

In [50]:
withConstraintTimesAll = report_times(deleteAll_times)

Verifying Constraints is ENABLED
Statistics for Insert with 10 rows, Verify Constraints set to: True
Verifying Constraints is ENABLED
Statistics for:  RepairTable
	MEDIAN3.705739974975586 ms
	MEAN  4.221343994140625 ms
	STDEV 1.2817323451635583 ms
Verifying Constraints is ENABLED
Statistics for:  MechTable
	MEDIAN1.24359130859375 ms
	MEAN  1.2331008911132812 ms
	STDEV 0.058039378213424316 ms
Verifying Constraints is ENABLED
Statistics for:  CarTable
	MEDIAN1.2421607971191406 ms
	MEAN  1.2804985046386719 ms
	STDEV 0.12511267950272062 ms


In [51]:
Table.RemoveAllTables(client, True) #True to wait for confirmation

Removing rows in Table  mechanics
Removing rows in Table  cars
Removing rows in Table  repairs
Removing rows in Table  mech_num_FKTable
Removing rows in Table  car_num_FKTable
Note:  25 records removed


In [52]:
Table.SetVerifyConstraints(False)
Table.UseFKTables(False)

In [53]:
BuildTables()
InsertData()

mechanics has complete FK to PK references (or none exist in table)
cars has complete FK to PK references (or none exist in table)
repairs has complete FK to PK references (or none exist in table)
PKvals from Insert:  [0] for Table:  mechanics
JSON string: {"mech_num": 0, "mech_name": "someName", "mech_salary": "someSalary", "mech_specialty": "someSpecialty"}
PKvals from Insert:  [0] for Table:  cars
JSON string: {"car_num": 0, "car_model": "someCar", "car_year": "someYear"}
PKvals from Insert:  [1] for Table:  mechanics
JSON string: {"mech_num": 1, "mech_name": "someName", "mech_salary": "someSalary", "mech_specialty": "someSpecialty"}
PKvals from Insert:  [1] for Table:  cars
JSON string: {"car_num": 1, "car_model": "someCar", "car_year": "someYear"}
PKvals from Insert:  [2] for Table:  mechanics
JSON string: {"mech_num": 2, "mech_name": "someName", "mech_salary": "someSalary", "mech_specialty": "someSpecialty"}
PKvals from Insert:  [2] for Table:  cars
JSON string: {"car_num": 2, "c

{'RepairTable': [0.6694793701171875,
  0.6506443023681641,
  0.5981922149658203,
  0.6132125854492188,
  0.6184577941894531,
  0.6170272827148438,
  0.6146430969238281,
  0.6010532379150391,
  0.5850791931152344,
  0.5900859832763672]}

In [54]:
deleteStartTime = time.time() #deleteStartTime/EndTime only use to give an overall time for the delete loop
deleteAll_times = DeleteData()
deleteEndTime = time.time()

[7, 7, 'someRepairDate']  exists in the following row (VerifyUniquePKval):
{'mech_num': 7, 'car_num': 7, 'repair_date': 'someRepairDate'}
****Removing record  [7, 7, 'someRepairDate']
[7]  exists in the following row (VerifyUniquePKval):
{'mech_num': 7, 'mech_name': 'someName', 'mech_salary': 'someSalary', 'mech_specialty': 'someSpecialty'}
****Removing record  [7]
[7]  exists in the following row (VerifyUniquePKval):
{'car_num': 7, 'car_model': 'someCar', 'car_year': 'someYear'}
****Removing record  [7]
[3, 3, 'someRepairDate']  exists in the following row (VerifyUniquePKval):
{'mech_num': 3, 'car_num': 3, 'repair_date': 'someRepairDate'}
****Removing record  [3, 3, 'someRepairDate']
[3]  exists in the following row (VerifyUniquePKval):
{'mech_num': 3, 'mech_name': 'someName', 'mech_salary': 'someSalary', 'mech_specialty': 'someSpecialty'}
****Removing record  [3]
[3]  exists in the following row (VerifyUniquePKval):
{'car_num': 3, 'car_model': 'someCar', 'car_year': 'someYear'}
****R

In [55]:
noConstraintTimesAll = report_times(deleteAll_times)

Verifying Constraints is DISABLED
Statistics for Insert with 10 rows, Verify Constraints set to: False
Verifying Constraints is DISABLED
Statistics for:  RepairTable
	MEDIAN1.2068748474121094 ms
	MEAN  1.4759063720703125 ms
	STDEV 0.6929920248288116 ms
Verifying Constraints is DISABLED
Statistics for:  MechTable
	MEDIAN1.1532306671142578 ms
	MEAN  1.3197898864746094 ms
	STDEV 0.4071002509196096 ms
Verifying Constraints is DISABLED
Statistics for:  CarTable
	MEDIAN1.1255741119384766 ms
	MEAN  1.2801170349121094 ms
	STDEV 0.3294625332360354 ms


In [56]:
benchdata.rcrd_data("Total Delete Time: {} seconds, 3 tables".format(deleteEndTime - deleteStartTime))
report_statistics(withConstraintTimesAll, noConstraintTimesAll)

Total Delete Time: 0.020561695098876953 seconds, 3 tables
Statistics for:  RepairTable


	With Constraints:		Without Constraints		Percent Difference, 10 rows
MEDIAN:	3.705739974975586 		1.2068748474121094 ms		207.05254839984195
MEAN:	4.221343994140625 		1.4759063720703125 ms		204.2452830188679
STDEV:	1.2817323451635583 		0.6929920248288116 ms		84.95629087220476


Statistics for:  MechTable


	With Constraints:		Without Constraints		Percent Difference, 10 rows
MEDIAN:	1.24359130859375 		1.1532306671142578 ms		7.835435187099442
MEAN:	1.2331008911132812 		1.3197898864746094 ms		6.051737842329648
STDEV:	0.058039378213424316 		0.4071002509196096 ms		-85.74322219592898


Statistics for:  CarTable


	With Constraints:		Without Constraints		Percent Difference, 10 rows
MEDIAN:	1.2421607971191406 		1.1255741119384766 ms		10.357975005295488
MEAN:	1.2804985046386719 		1.2801170349121094 ms		12.102361618118156
STDEV:	0.12511267950272062 		0.3294625332360354 ms		-62.025217777013




In [57]:
benchdata.close()
del benchdata

Finally, remove the leftover tables

In [58]:
Table.RemoveAllTables(client, True) #True to wait for confirmation

Removing rows in Table  mechanics
Removing rows in Table  cars
Removing rows in Table  repairs
Removing rows in Table  mech_num_FKTable
Removing rows in Table  car_num_FKTable
Note:  15 records removed


In [59]:
client.close()